In [3]:
import pandas as pd

In [ ]:
#loading Phishing urls
dt1=pd.read_csv("2.online-valid.csv")
dt1.head()
#collecting 5,000 phishing urls randomly
phisurl=dt1.sample(n=5000,random_state=12).copy()
phisurl=phisurl.reset_index(drop=True)
phisurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6514946,http://confirmprofileaccount.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-04-19T11:06:55+00:00,yes,2020-04-19T13:42:41+00:00,yes,Other
1,4927651,http://www.marreme.com/MasterAdmin/04mop.html,http://www.phishtank.com/phish_detail.php?phis...,2017-04-04T19:35:54+00:00,yes,2017-05-03T23:00:42+00:00,yes,Other
2,5116976,http://modsecpaststudents.com/review/,http://www.phishtank.com/phish_detail.php?phis...,2017-07-25T18:48:30+00:00,yes,2017-07-28T16:01:36+00:00,yes,Other
3,6356131,https://docs.google.com/forms/d/e/1FAIpQLScL6L...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-13T20:13:37+00:00,yes,2020-01-17T01:55:38+00:00,yes,Other
4,6535965,https://oportunidadedasemana.com/americanas//?...,http://www.phishtank.com/phish_detail.php?phis...,2020-04-29T00:01:03+00:00,yes,2020-05-01T10:55:35+00:00,yes,Other


\

In [ ]:
phisurl.shape

(5000, 8)

In [ ]:
#loading legitamte urls
dt2 = pd.read_csv("/content/Benign_list_big_final .csv")
dt2.columns = ['URLs']
dt2.head()
legiurl = dt2.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()
legiurl

,URLs
0,http://grantland.com/hollywood-prospectus/seco...
1,http://correios.com.br/para-voce/consultas-e-s...
2,https://ifttt.com/recipes/1552-if-i-post-a-pic...
3,http://babal.net/books/view/118/%D9%82%D8%B5%D...
4,http://getpocket.com/signup?mode=socialmode&t=...
...,...
4995,http://mic.com/articles/116574/lena-dunham-on-...
4996,http://buzzfil.net/article/5292/buzz/quelle-em...
4997,http://xvideo-jp.com/archives/category/%e3%81%...
4998,http://thenextweb.com/au/2013/08/31/25-mbps-br...


In [ ]:
legiurl.shape


(5000, 1)

In [ ]:
!pip install python-whois


In [ ]:

from datetime import datetime
import ipaddress
import urllib
import urllib.request
import whois
from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
import requests


In [ ]:
#1.domain
def getDomain(url):
    domain = urlparse(url).netloc
    if domain.startswith("www."):
        domain = domain[4:]
    return domain


In [ ]:
# We dropped domain in training data

# 2.IP
def havingIP(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip

In [ ]:

#3. @ sign
def haveAtSign(url):
    if '@' in url:
        at = 1
    else:
        at = 0
    return at

In [ ]:
#4. URL length if longer than 54 characters mark as phishing
def urlLength(url):
    if len(url) < 54:
        return 0 #safe/normal
    elif len(url) <= 75:
        return-1  # Suspicious
    else:
        return 1 #Phishing


In [ ]:

#5. Return URL depth
from urllib.parse import urlparse

def urlDepth(url):
    path = urlparse(url).path
    return len([p for p in path.split('/') if p])


In [ ]:
#6. Checking number of redirections, if greater than 6 mark as phishing
import requests
def redirection(url):
    try:
        response = requests.get(url, timeout=5, allow_redirects=True)
        # Count how many redirects occurred
        num_redirects = len(response.history)
        return 1 if num_redirects > 6 else 0
    except:
        # If the request fails, assume suspicious (or handle separately)
        return 1


In [ ]:

#7. Check if url is shortened

def urlShort(url):
    shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                          r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                          r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                          r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                          r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                          r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                          r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                          r"tr\.im|link\.zip\.net"
    match = re.search(shortening_services, url)
    if match:
        return 1
    else:
        return 0

In [ ]:

#8. Check if prefix or suffix is used in domain name

def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1
    else:
        return 0

In [ ]:
#9. Check if https is in the url
def httpsInUrl(url):
    domain = urlparse(url).netloc
    if 'https' in domain:
        return 1
    else:
        return 0

In [ ]:

#10. website traffic

def web_traffic(url):
    try:
        rank_checker_response = requests.post("https://www.checkpagerank.net/index.php", {"name": url.domain})

        global_rank = int(re.findall(r"Global Rank: ([0-9]+)", rank_checker_response.text)[0])
        if global_rank > 0 and global_rank < 100000:
            return 1 #popular site
        return -1
    except:
        return -1

In [ ]:
#11. domain age


def domainAge(domain_name):
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date

    # Handle cases where creation_date or expiration_date might be lists
    if isinstance(creation_date, list):
        creation_date = creation_date[0]
    if isinstance(expiration_date, list):
        expiration_date = expiration_date[0]

    if (isinstance(creation_date, str) or isinstance(expiration_date, str)):
        try:
            creation_date = datetime.strptime(creation_date, '%Y-%m-%d')
            expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")
        except:
            return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain / 30) < 6):
            age = 1
        else:
            age = 0
    return age

In [ ]:
from datetime import datetime
import whois
def domainExpiry(url):
    try:
        domain_info = whois.whois(url)
        expiration_date = domain_info.expiration_date

        # If multiple dates are returned, pick the first one
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]

        # Handle missing expiration date
        if expiration_date is None:
            return 1  # phishing

        # If expiration_date is a string, try converting
        if isinstance(expiration_date, str):
            try:
                expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")
            except ValueError:
                return 1  # phishing

        # Calculate days until expiration
        today = datetime.now()
        days_to_expiry = (expiration_date - today).days

        # If less than 180 days (6 months), flag as phishing
        if days_to_expiry < 180:
            return 1  # phishing
        else:
            return 0  # legitimate

    except Exception as e:
        # Any error (domain not found, no data, etc.) → flag as phishing
        print(f"WHOIS lookup failed for {url}: {e}")
        return 1  # phishing



In [ ]:
#13. iFrame

def iframe(response):
    if response == "":
        return 1
    else:
        if re.findall(r"[<iframe>|<frameBorder>]", response.text):
            return 1
        else:
            return 0


In [ ]:
#14.Mouse Over

def mouseOver(response):
    if response == "":
        return 1
    else:
        if re.findall("<script>.+onmouseover.+</script>", response.text):
            return 1
        else:
            return 0


In [ ]:

#15. Right Click (disabled)

def rightClick(response):
    if response == "":
        return 1
    else:
        if re.findall(r"event.button ?== ?2", response.text):
            return 1
        else:
            return 0

In [ ]:

#16. Check how many times a website has been redirected

def forwarding(response):
    if response == "":
        return 1
    else:
        if len(response.history) <= 2:
            return 0
        elif len(response.history) <= 4:
            return 1
        else:
            return 0

In [ ]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(urlLength(url))
  features.append(urlDepth(url))
  features.append(redirection(url))
  features.append(httpsInUrl(url))
  features.append(urlShort(url))
  features.append(prefixSuffix(url))

  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  features.append(web_traffic(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainExpiry(domain_name))

  # HTML & Javascript based features (4)
  try:
    response = requests.get(url)
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)

  return features


In [ ]:
legiurl.shape

(5000, 1)

In [ ]:
import socket
from tqdm import tqdm
import time

In [ ]:
#extracting features for legitimate
socket.setdefaulttimeout(10)
time.sleep(1)

legi_features = []
label = 0

for index, row in tqdm(legiurl.iterrows(), total=min(5000, len(legiurl))):
    if index >= 5000:
        break

    url = row['URLs']
    try:
        features = featureExtraction(url, label)
        legi_features.append(features)

    except Exception as e:
        print(f"Error processing {url}: {e}")
    continue

  0%|          | 1/5000 [00:00<56:25,  1.48it/s]

WHOIS lookup failed for {
  "domain_name": "GRANTLAND.COM",
  "registrar": "CSC Corporate Domains, Inc.",
  "registrar_url": "http://cscdbs.com",
  "reseller": null,
  "whois_server": "whois.corporatedomains.com",
  "referral_url": null,
  "updated_date": "2025-05-12 05:02:05",
  "creation_date": "1997-06-12 04:00:00",
  "expiration_date": "2026-06-11 04:00:00",
  "name_servers": [
    "NS1.WORDPRESS.COM",
    "NS2.WORDPRESS.COM",
    "NS3.WORDPRESS.COM"
  ],
  "status": "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
  "emails": "domainabuse@cscglobal.com",
  "dnssec": "unsigned",
  "name": null,
  "org": null,
  "address": null,
  "city": null,
  "state": null,
  "registrant_postal_code": null,
  "country": null
}: expected string or bytes-like object, got 'WhoisCom'
WHOIS lookup failed for {
  "domain_name": "correios.com.br",
  "registrant_name": "Empresa Brasileira de Correios e Tel�grafos",
  "registrant_id": "34.028.316/0001-03",
  "country": "BR",
  "

  0%|          | 2/5000 [00:05<4:27:08,  3.21s/it]

WHOIS lookup failed for {
  "domain_name": "IFTTT.COM",
  "registrar": "GoDaddy.com, LLC",
  "registrar_url": "http://www.godaddy.com",
  "reseller": null,
  "whois_server": "whois.godaddy.com",
  "referral_url": null,
  "updated_date": "2025-07-15 14:49:51",
  "creation_date": "2009-07-14 17:19:46",
  "expiration_date": "2026-07-14 17:19:46",
  "name_servers": [
    "NS-1400.AWSDNS-47.ORG",
    "NS-1614.AWSDNS-09.CO.UK",
    "NS-425.AWSDNS-53.COM",
    "NS-676.AWSDNS-20.NET"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientRenewProhibited https://icann.org/epp#clientRenewProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited"
  ],
  "emails": "abuse@godaddy.com",
  "dnssec": "unsigned",
  "name": null,
  "org": null,
  "address": null,
  "city": null,
  "state": null,
  "registrant_postal_code": null,
  "country": null
}:

  0%|          | 4/5000 [00:06<1:58:20,  1.42s/it]

...

WHOIS lookup failed for {
  "domain_name": "MIC.COM",
  "registrar": "Amazon Registrar, Inc.",
  "registrar_url": [
    "http://registrar.amazon.com",
    "https://registrar.amazon.com"
  ],
  "reseller": null,
  "whois_server": "whois.registrar.amazon",
  "referral_url": null,
  "updated_date": "2023-01-09 08:15:32",
  "creation_date": "1993-09-30 04:00:00",
  "expiration_date": "2030-09-29 04:00:00",
  "name_servers": [
    "NS-1532.AWSDNS-63.ORG",
    "NS-1871.AWSDNS-41.CO.UK",
    "NS-326.AWSDNS-40.COM",
    "NS-566.AWSDNS-06.NET"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited"
  ],
  "emails": [
    "trustandsafety@support.aws.com",
    "0940b316-765e-41af-b63b-b2ea08cc2cc7@identity-protect.org"
  ],
  "dnssec": "unsigned",
  "name": "On behalf of mic.com owner",
  "org": "Identity Pro

  0%|          | 16/5000 [00:28<1:14:27,  1.12it/s]

WHOIS lookup failed for {
  "domain_name": "olx.in",
  "registrar": "CSC Corporate Domains, Inc.",
  "registrar_url": "www.cscglobal.com",
  "registrar_iana": "299",
  "updated_date": "2025-03-31 14:24:57.852000",
  "creation_date": "2006-02-22 11:21:13.346000",
  "expiration_date": "2035-02-22 11:21:13.346000",
  "name_servers": [
    "ns-1665.awsdns-16.co.uk",
    "ns-859.awsdns-43.net",
    "ns-15.awsdns-01.com",
    "ns-1346.awsdns-40.org"
  ],
  "organization": "OLX India B.V.",
  "state": null,
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited"
  ],
  "emails": "tldsupport@cscinfo.com",
  "country": "NL",
  "dnssec": "unsigned"
}: expected string or bytes-like object, got 'WhoisIn'


  0%|          | 18/5000 [00:31<1:40:34,  1.21s/it]

WHOIS lookup failed for {
  "domain_name": "KATPROXY.COM",
  "registrar": "DYNADOT LLC",
  "registrar_url": "http://www.dynadot.com",
  "reseller": null,
  "whois_server": "whois.dynadot.com",
  "referral_url": null,
  "updated_date": "2024-10-17 22:24:19",
  "creation_date": "2019-10-11 18:49:53",
  "expiration_date": "2025-10-11 18:49:53",
  "name_servers": [
    "NS1.PARKLOGIC.COM",
    "NS2.PARKLOGIC.COM"
  ],
  "status": "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
  "emails": "abuse@dynadot.com",
  "dnssec": "unsigned",
  "name": "REDACTED FOR PRIVACY",
  "org": "Super Privacy Service LTD c/o Dynadot",
  "address": "PO Box 701",
  "city": "San Mateo",
  "state": "California",
  "registrant_postal_code": "94401",
  "country": "US"
}: expected string or bytes-like object, got 'WhoisCom'
WHOIS lookup failed for {
  "domain_name": "allegro.pl",
  "name_servers": [
    "dns1.allegro.pl",
    "dns2.allegro.pl",
    "dns3.allegro.pl",
    "dns4.allegro.pl"


In [ ]:
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic',
                      'Domain_Age', 'Domain_Expiry', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards','Multiple_Subdomains','ssl_feature','Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

In [ ]:
# Storing the extracted legitimate URLs fatures to csv file
legitimate.to_csv('legitimate.csv', index= False)

In [ ]:
legitimate = pd.DataFrame(legi_features,columns=feature_names)
legitimate.head()

,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards
0,0,0,1,4,0,0,0,0,1,0,1,1,0,0,0,0
1,0,0,1,6,1,0,0,0,1,0,1,1,0,0,0,0
2,0,0,1,3,0,0,0,0,1,0,1,1,0,0,0,0
3,0,0,1,9,0,0,0,0,1,0,1,1,0,0,1,0
4,0,0,1,5,0,0,0,0,1,0,1,1,0,0,0,0


In [ ]:
#Extracting the feautres & storing them in a list
import socket
from tqdm import tqdm
import time
import pandas as pd
socket.setdefaulttimeout(10)
time.sleep(1)

phish_features = []
label = 1

for index, row in tqdm(phisurl.iterrows(), total=min(5000, len(phisurl))):
    if index >= 5000:
        break
    url = row['url']
    try:
        features = featureExtraction(url, label)
        phish_features.append(features)
    except Exception as e:
        print(f"Error processing {url}: {e}")
    continue



In [ ]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic',
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards','label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,label
0,confirmprofileaccount.com,0,0,0,2,0,0,1,0,1,-1,1,1,1,1,1,1,1
1,marreme.com,0,0,0,4,0,0,0,0,0,-1,0,0,1,0,0,0,1
2,modsecpaststudents.com,0,0,0,3,0,0,0,0,0,-1,0,1,1,0,0,0,1
3,docs.google.com,0,0,1,7,0,0,0,0,0,-1,0,1,1,0,0,0,1
4,oportunidadedasemana.com,0,0,1,4,1,0,0,0,1,-1,1,1,1,1,1,1,1


In [ ]:
phishing.to_csv('phishing.csv', index= False)

In [ ]:
legitimate=pd.read_csv("legi_features.csv")
legitimate.head()

In [7]:
#Concatenating the dataframes into one
legitimate_df = pd.read_csv("legitimate.csv")
phishing_df = pd.read_csv("phishing.csv")
urldata = pd.concat([legitimate_df, phishing_df]).reset_index(drop=True)
display(urldata.head())

,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Domain,label
0,0,0,1,4,0,0,0,0,1,0,1,1,0,0,0,0,NaN,NaN
1,0,0,1,6,1,0,0,0,1,0,1,1,0,0,0,0,NaN,NaN
2,0,0,1,3,0,0,0,0,1,0,1,1,0,0,0,0,NaN,NaN
3,0,0,1,9,0,0,0,0,1,0,1,1,0,0,1,0,NaN,NaN
4,0,0,1,5,0,0,0,0,1,0,1,1,0,0,0,0,NaN,NaN
